In [ ]:
import pandas as pd

In [ ]:

df = pd.read_parquet('../data/processed/bird-whisperer/train_downsampled.parquet')


# Remove augmentations

In [ ]:
augmented_df = df[df['augmented'] == True]
first_augmented = augmented_df.sort_values(by=["id", "species"]).groupby("id").first().reset_index()
non_augmented_df = df[df['augmented'] == False]
result_df = pd.concat([non_augmented_df, first_augmented])
result_df.sort_values(by=["id"])
result_df.shape

# Downsampling

In [ ]:
target_count = 200
species_counts = result_df['species'].value_counts()
majority_species = species_counts[species_counts > target_count].index

downsampled_data = []
for species in majority_species:
    species_data = result_df[result_df['species'] == species]
    sampled_data = species_data.sample(n=target_count, random_state=42)  
    downsampled_data.append(sampled_data)

minority_data = result_df[~result_df['species'].isin(majority_species)]
downsampled_data.append(minority_data)
final_data = pd.concat(downsampled_data)

final_data.shape
final_data.to_parquet("../data/processed/bird-whisperer/downsampled_train.parquet", index=False)

# Only allow classes above 100 non-augmented records

In [ ]:
import pandas as pd

df_train = pd.read_parquet("/exchange/dspro01/HSLU.DSPRO1.BirdDetection/data/processed/bird-whisperer-denoised/train.parquet")
df_test = pd.read_parquet("/exchange/dspro01/HSLU.DSPRO1.BirdDetection/data/processed/bird-whisperer-denoised/test.parquet")

species_counts = df_train[df_train['augmented'] == False].groupby('species').size()
species_to_keep = species_counts[species_counts >= 100].index

filtered_df_train = df_train[df_train['species'].isin(species_to_keep)]
filtered_df_test = df_test[df_test['species'].isin(species_to_keep)]

print(f"Filtered Training Dataset Shape: {filtered_df_train.shape}")
print(f"Filtered Test Dataset Shape: {filtered_df_test.shape}")

filtered_df_train.to_parquet("/exchange/dspro01/HSLU.DSPRO1.BirdDetection/data/processed/bird-whisperer-denoised/train_cutoff.parquet")
filtered_df_test.to_parquet("/exchange/dspro01/HSLU.DSPRO1.BirdDetection/data/processed/bird-whisperer-denoised/test_cutoff.parquet")
